In [151]:
import requests
import bs4
import json
import sys
sys.path.insert(0, '../')  # добавляем путь к папке верхнего уровня
sys.path.insert(0, '.../') 

from OTCParse.parser_requests import requests_info, make_request
from OTCParse.filters import filters_dict

In [152]:
import pandas as pd
import re
import numpy as np

In [ ]:
## from sentence_transformers import SentenceTransformer
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence')

In [154]:
prod = pd.read_json('products2.json')
prod["sys_embbedings"] = products["sys_embbedings"].apply(lambda x: np.array(x))

In [155]:
def _calc_reliability(x):
    if x == "высокая":
        return 1.0
    if x == "нет информации":
        return 0.9
    if x == "средняя":
        return 0.95
    if x == "низкая":
        return 0.85
prod['reliability_float'] = products['reliability'].apply(_calc_reliability)
prod.head()

,index,name,okpd-2,price,shipping_region,ogrn,inn,sys_float_price,sys_simple_name,sys_embbedings,description,reliability,reliability_float
0,0,Модуль памяти Kingston VALUERAM KVR26N19S6/8 D...,26.20.1,3 530 ₽/шт,Все регионы,1153443018630,3459063755,3530.0,модуль памяти,"[-0.5423593521, -1.4277704954000001, 0.1996717...",Модуль памяти Kingston VALUERAM KVR26N19S6/8 D...,высокая,1.0
1,1,Устройство охлаждения(кулер) Aerocool Verkho A...,26.20.1,650 ₽/шт,Все регионы,1153443018630,3459063755,650.0,устройство охлаждениякулер,"[-1.1588402987, -1.0989841223, -0.5186546445, ...",Устройство охлаждения(кулер) Aerocool Verkho A...,высокая,1.0
2,2,Блок питания Xilence Red Wings XN052 500W,26.20.1,3 590 ₽/шт,Все регионы,1153443018630,3459063755,3590.0,блок питания,"[-0.4660938978, -0.622358799, -0.0173774697000...",Блок питания Xilence Red Wings XN052 500W,высокая,1.0
3,3,Клавиатура+мышь проводная SVEN KB-S330C черный,26.20.1,1 370 ₽/шт,Все регионы,1153443018630,3459063755,1370.0,клавиатурамышь проводная,"[-1.1191068888, -0.5200283527, -0.096627779300...",Клавиатура+мышь проводная SVEN KB-S330C черный,высокая,1.0
4,4,"Термопаста Cooler Master IC-Value V1 шприц, 4....",26.20.1,1 810 ₽/шт,Все регионы,1153443018630,3459063755,1810.0,термопаста,"[-1.2554225922, -0.44866523150000004, 0.397536...","Термопаста Cooler Master IC-Value V1 шприц, 4....",высокая,1.0


In [156]:
supp = pd.read_json('suppliers.json')
supp.head()

,index,ogrn,reliability,reg_date,main_okved,additionalokveds,authorized_capital,workers,address,claimant,defendant,active_debts,completed_debts,status,RNP,facts,company_name
0,0,1072721007721,высокая,26.04.2007,68.20,"[46.43.1, 46.43.3, 46.45.1, 46.49.33, 46.51, 4...",20000.0,8.0,край Хабаровский г Хабаровск ул Льва Толстого ...,17,1,0.0,0,1.0,0.0,"{'good': '24', 'normal': '6', 'bad': '0'}","ООО СК ""ЛУННЫЙ СВЕТ"""
1,1,1097746252548,средняя,27.04.2009,62.01,"[58, 61.1, 61.10.1, 61.10.9, 62.02, 62.09, 63....",20005040.0,123.0,"г Москва ул Житная Д. 10, ЭТАЖ 1, ПОМЕЩ. VI - ...",57,73,0.0,4,1.0,0.0,"{'good': '17', 'normal': '6', 'bad': '8'}","АО ""ОТС"""
2,2,318619600041146,None,06.03.2018,43.31,"[13.92, 13.92.1, 13.92.2, 13.99, 28.23, 33.20,...",NaN,NaN,обл Ростовская,0,0,0.0,0,1.0,0.0,None,ИП АСКАЛЕПОВ С. Н.
3,3,1207700142616,высокая,27.03.2020,18.13,"[18.12, 46.14.1, 46.15, 46.15.4, 46.15.9, 46.1...",10000.0,6.0,"г Москва проезд Нагатинский 1-й Д. 6, СТР. 1, ...",455,0,0.0,0,1.0,0.0,"{'good': '21', 'normal': '4', 'bad': '4'}","ООО ""ИНТЕРПАКС"""
4,4,1027402933040,высокая,30.12.2002,46.49,"[22.22, 22.23, 22.29, 25.99, 43.21, 43.99, 46....",8400.0,53.0,обл Челябинская г Челябинск ул Телеграфная Д. 44,36,2,0.0,1,1.0,0.0,"{'good': '19', 'normal': '8', 'bad': '2'}","ООО ""КАНЦБЮРО"""


In [157]:
all_ogrns = set(supp["ogrn"].values) & set(prod["ogrn"].values)
all_ogrns = list(all_ogrns)
mask = prod["ogrn"].apply(lambda x: x in all_ogrns)
prod = prod[mask]
mask = supp["ogrn"].apply(lambda x: x in all_ogrns)
supp = supp[mask]

In [182]:
from Levenshtein import distance
import numpy as np
from sentence_transformers import SentenceTransformer
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence')

def _calc_cosinus(a, b):
    return np.sum(a*b)/(np.sum(a*a)*np.sum(b*b))**0.5

def calc_occurrences_single(x, responses):
    lem_responses = [morph.parse(word)[0].normal_form for word in responses]
    matchings = [resp.lower() in x.lower() for resp in responses]
    return np.mean(matchings)**0.5

def calc_metric_occurrences(df, response):
    responses = response.split()
    score = df.apply(lambda x: calc_occurrences_single(x["name"], responses) , axis = 1)
    return score

def calc_metric_lev(df, response = ""):
    score = 1-df.apply(lambda x: distance(x["simple_name"].lower(), response.lower())/len(response), axis = 1)
    return score

def calc_metric_nlp(df, response):
    embbedings = df["sys_embbedings"].values
    target = model.encode(response)
    score = [_calc_cosinus(emb, target) for emb in embbedings]
 
    return score

def ranking_by_metric(df,response, metric, topk = 10, page = 0):
    df_new = df.copy()
    df_new["similarity_score"] = metric(df, response)
    if topk == -1:
        return df_new
    else:
        df_new = df_new.sort_values(["similarity_score"], ascending = [False])
        return df_new[page*topk:(page+1)*topk]

# def ranking(df,response, topk = 5):
#     df_new = df.copy()
#     df_new["similarity_score"] = metric(df, response)
#     df_new = df_new.sort_values(["similarity_score"], ascending = [False])
#     return df_new[:topk]

No sentence-transformers model found with name /Users/kirill/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased-sentence. Creating a new one with MEAN pooling.


In [183]:
def _max_sim(x):
    return np.max(x["similarity_score"].values)

class Ranker():
    def __init__(self, prod, supp):
        self.prod = prod
        self.supp = supp
    
    @staticmethod
    def get_responses(responses):
        responses = responses.split('|')
        responses = [resp for resp in responses if len(resp)>0]
        return responses
 
    def get_ranking_data(self, responses, page = 0, elemnts_per_page = 10, searc_by = "product"):
        if searc_by == "customer":
            responses = self.get_responses(responses)
            supp_c = self.supp.copy()
            sum_col = []
            for resp in responses:
                df = ranking_by_metric(self.prod, responses, calc_metric_nlp, -1, page)
                prod = self.prod.copy()
                temp = df.groupby('ogrn', sort=False).apply(_max_sim)
                supp_c[f'sys_{resp}'] = temp[supp_c['ogrn'].values].values
                sum_col.append(f'sys_{resp}')
            supp_c["mean_rating"] = supp_c[sum_col].mean(axis = 1)
            supp_c = supp_c.sort_values(["mean_rating"], ascending = [False])
            return supp_c[page*elemnts_per_page:(page+1)*elemnts_per_page]
        else: 
            response = responses
            df = ranking_by_metric(self.prod, responses, calc_metric_nlp, elemnts_per_page, page)
            return df
        
response = "ноутбук"
r = Ranker(prod, supp)
df = r.get_ranking_data(response, page = 0, elemnts_per_page = 10, searc_by = "product")
        

    

In [ ]:
df

In [201]:
df = ranking_by_metric(prod, "Ноутбук", calc_metric_nlp, 10, 0)


In [202]:
df = ranking_by_metric(prod, "Ноутбук", calc_metric_occurrences, 10, 0)

In [207]:
prod

,index,name,okpd-2,price,shipping_region,ogrn,inn,sys_float_price,sys_simple_name,sys_embbedings,description,reliability,reliability_float
3655,0,Зажим для бумаг 25мм серебро BC_059165,22.29.25.000,11.0,край Хабаровский,1072721007721,2721148656,11.0,зажим для бумаг мм серебро,"[0.29026964310000003, -0.6288420558000001, -0....",Зажимы 25 мм представлены в актуальном серебря...,высокая,1.00
3656,1,"Магнит д-доски 3,5см 1шт deVENTE Balls 6021000",13.92.29.120,11.0,край Хабаровский,1072721007721,2721148656,11.0,магнит ддоски см шт,"[-0.9036120176, -1.1413147449, 0.1704514325, -...","Магнит д-доски 3,5см 1шт deVENTE Balls 6021000",высокая,1.00
3657,2,Easyprint CN046AE/№951XL Картридж (IH-046) №95...,26.20.16.110,12.0,г Москва,1097746252548,7708699893,12.0,картридж,"[-0.7557550073, -0.8466399312, -0.240803718600...",Производитель:EasyPrint;Модель:CN046AE;Тип обо...,средняя,0.95
3658,3,Easyprint CN048AE/№951XL Картридж (IH-048) №95...,26.20.16.110,12.0,г Москва,1097746252548,7708699893,12.0,картридж,"[-0.7557550073, -0.8466399312, -0.240803718600...",Производитель:EasyPrint;Модель:CN048AE;Тип обо...,средняя,0.95
3659,4,Xerox C7000EUD Документация VERSALINK C7000,26.20.16.110,12.0,г Москва,1097746252548,7708699893,12.0,документация,"[-0.31082442400000004, 0.0472454168, -0.029269...",Производитель:Xerox;ширина(см):18.0000;Страна ...,средняя,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58347,81305,Виниловый ламинат Alpine Floor Intense ECO 9-3...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771461730000002, -0.7213525772, -0.76502...",Виниловый ламинат Alpine Floor Intense ECO 9-3...,низкая,0.85
58348,81306,Виниловый ламинат Alpine Floor Intense ECO 9-7...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771461730000002, -0.7213525772, -0.76502...",Виниловый ламинат Alpine Floor Intense ECO 9-7...,низкая,0.85
58349,81307,Виниловый ламинат Alpine Floor Intense ECO 9-1...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771461730000002, -0.7213525772, -0.76502...",Виниловый ламинат Alpine Floor Intense ECO 9-1...,низкая,0.85
58350,81308,Виниловый ламинат Alpine Floor Intense ECO 9-6...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771496, -0.7213516235, -0.7650208473, -0...",Виниловый ламинат Alpine Floor Intense ECO 9-6...,низкая,0.85


In [122]:
supp_c

,ogrn,reliability,reg_date,main_okved,additionalokveds,authorized_capital,workers,address,claimant,defendant,active_debts,completed_debts,status,RNP,facts,rating
0,1072721007721,высокая,26.04.2007,68.20,"['46.43.1', '46.43.3', '46.45.1', '46.49.33', ...",20000.0,8.0,край Хабаровский г Хабаровск ул Льва Толстого ...,17,1,0.0,0,1.0,0.0,"{'good': '24', 'normal': '6', 'bad': '0'}",0.859996
1,1097746252548,средняя,27.04.2009,62.01,"['58', '61.1', '61.10.1', '61.10.9', '62.02', ...",20005040.0,123.0,"г Москва ул Житная Д. 10, ЭТАЖ 1, ПОМЕЩ. VI - ...",57,73,0.0,4,1.0,0.0,"{'good': '17', 'normal': '6', 'bad': '8'}",0.885598
2,318619600041146,None,06.03.2018,43.31,"['13.92', '13.92.1', '13.92.2', '13.99', '28.2...",NaN,NaN,обл Ростовская,0,0,0.0,0,1.0,0.0,None,0.782044
3,1207700142616,высокая,27.03.2020,18.13,"['18.12', '46.14.1', '46.15', '46.15.4', '46.1...",10000.0,6.0,"г Москва проезд Нагатинский 1-й Д. 6, СТР. 1, ...",455,0,0.0,0,1.0,0.0,"{'good': '21', 'normal': '4', 'bad': '4'}",0.840267
4,1027402933040,высокая,30.12.2002,46.49,"['22.22', '22.23', '22.29', '25.99', '43.21', ...",8400.0,53.0,обл Челябинская г Челябинск ул Телеграфная Д. 44,36,2,0.0,1,1.0,0.0,"{'good': '19', 'normal': '8', 'bad': '2'}",0.771090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488,319265100070402,None,27.05.2019,33.20,"['28.15', '28.22.7', '43.99.5']",NaN,NaN,край Ставропольский,0,0,0.0,0,1.0,0.0,None,0.431491
1489,1113801014414,высокая,11.11.2011,81.22,"['18.1', '18.12', '41.20', '46.1', '46.4', '47...",10000.0,5.0,"обл Иркутская г Ангарск кв-л 52 Д.6, ОФИС 18А",0,0,0.0,0,1.0,0.0,"{'good': '23', 'normal': '6', 'bad': '1'}",0.717064
1490,319732500004624,None,28.01.2019,46.90,['41.20'],NaN,NaN,обл Ульяновская,0,0,0.0,0,0.0,0.0,None,0.717064
1491,1022901218010,высокая,30.11.2002,84.23.4,"['01.13.11', '01.13.12', '01.13.31', '01.41.11...",NaN,NaN,обл Архангельская р-н Вельский д Горка-Муравье...,37,31,2.0,1,1.0,0.0,"{'good': '12', 'normal': '5', 'bad': '0'}",0.662477


In [92]:
len(prod["ogrn"].drop_duplicates())

1493

In [86]:
a = set([1,2,3])
b = set([2,3,4])
a&all_ogrnsb

{2, 3}

In [93]:
len(prod)

54696

In [208]:
len(supp)

1493

In [209]:
prodх

,index,name,okpd-2,price,shipping_region,ogrn,inn,sys_float_price,sys_simple_name,sys_embbedings,description,reliability,reliability_float
3655,0,Зажим для бумаг 25мм серебро BC_059165,22.29.25.000,11.0,край Хабаровский,1072721007721,2721148656,11.0,зажим для бумаг мм серебро,"[0.29026964310000003, -0.6288420558000001, -0....",Зажимы 25 мм представлены в актуальном серебря...,высокая,1.00
3656,1,"Магнит д-доски 3,5см 1шт deVENTE Balls 6021000",13.92.29.120,11.0,край Хабаровский,1072721007721,2721148656,11.0,магнит ддоски см шт,"[-0.9036120176, -1.1413147449, 0.1704514325, -...","Магнит д-доски 3,5см 1шт deVENTE Balls 6021000",высокая,1.00
3657,2,Easyprint CN046AE/№951XL Картридж (IH-046) №95...,26.20.16.110,12.0,г Москва,1097746252548,7708699893,12.0,картридж,"[-0.7557550073, -0.8466399312, -0.240803718600...",Производитель:EasyPrint;Модель:CN046AE;Тип обо...,средняя,0.95
3658,3,Easyprint CN048AE/№951XL Картридж (IH-048) №95...,26.20.16.110,12.0,г Москва,1097746252548,7708699893,12.0,картридж,"[-0.7557550073, -0.8466399312, -0.240803718600...",Производитель:EasyPrint;Модель:CN048AE;Тип обо...,средняя,0.95
3659,4,Xerox C7000EUD Документация VERSALINK C7000,26.20.16.110,12.0,г Москва,1097746252548,7708699893,12.0,документация,"[-0.31082442400000004, 0.0472454168, -0.029269...",Производитель:Xerox;ширина(см):18.0000;Страна ...,средняя,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58347,81305,Виниловый ламинат Alpine Floor Intense ECO 9-3...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771461730000002, -0.7213525772, -0.76502...",Виниловый ламинат Alpine Floor Intense ECO 9-3...,низкая,0.85
58348,81306,Виниловый ламинат Alpine Floor Intense ECO 9-7...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771461730000002, -0.7213525772, -0.76502...",Виниловый ламинат Alpine Floor Intense ECO 9-7...,низкая,0.85
58349,81307,Виниловый ламинат Alpine Floor Intense ECO 9-1...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771461730000002, -0.7213525772, -0.76502...",Виниловый ламинат Alpine Floor Intense ECO 9-1...,низкая,0.85
58350,81308,Виниловый ламинат Alpine Floor Intense ECO 9-6...,22.23.15.000,2590.0,обл Московская,1195081076739,5047234290,2590.0,виниловый ламинат,"[-0.20771496, -0.7213516235, -0.7650208473, -0...",Виниловый ламинат Alpine Floor Intense ECO 9-6...,низкая,0.85
